In [ ]:
import numpy as np

import cv2

import os

# neural net
import torch
import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt
%matplotlib inline

## load image

In [ ]:
PATH = os.getcwd()
PATH

In [ ]:
# image path
img_path = PATH + '/data/boomberman.jpg'

# load color image 
bgr_img = cv2.imread(img_path)

# convert to grayscale
gray_img = cv2.cvtColor(bgr_img, cv2.COLOR_BGR2GRAY)

# normalize, rescale entries to lie in [0,1]
gray_img = gray_img.astype("float32")/255

# plot image
plt.imshow(gray_img, cmap='gray')
plt.show()

## define and visualize the filters

In [ ]:
def get_filters(base_filter):
    
    filter_1 = base_filter
    filter_2 = - base_filter
    filter_3 = base_filter.T
    filter_4 = - filter_3
    
    filters = np.array([ filter_1, filter_2, filter_3, filter_4 ])
    
    return filters

def plot_filters(filters):
    
    fig = plt.figure(figsize=(10, 5))
    for i in range(4):
        ax = fig.add_subplot(1, 4, i+1, xticks=[], yticks=[])
        ax.imshow(filters[i], cmap='gray')
        ax.set_title('Filter %s' % str(i+1))
        width, height = filters[i].shape
        for x in range(width):
            for y in range(height):
                ax.annotate(str(filters[i][x][y]), xy=(y,x),
                            horizontalalignment='center',
                            verticalalignment='center',
                            color='white' if filters[i][x][y]<0 else 'black')
                
    return fig

# helper function for visualizing the output of a given layer
# default number of filters is 4
def viz_layer(layer, n_filters= 4):
    fig = plt.figure(figsize=(20, 20))
    
    for i in range(n_filters):
        ax = fig.add_subplot(1, n_filters, i+1, xticks=[], yticks=[])
        # grab layer outputs
        ax.imshow(np.squeeze(layer[0,i].data.numpy()), cmap='gray')
        ax.set_title('Output %s' % str(i+1))

In [ ]:
# filters
filter_A = np.array([
    [-1, 1], 
    [-1, 1]
])

filter_B = np.array([
    [-1, -1, 1, 1], 
    [-1, -1, 1, 1], 
    [-1, -1, 1, 1], 
    [-1, -1, 1, 1]
])

print('Filter A shape: ', filter_A.shape)
print('Filter B shape: ', filter_B.shape)

In [ ]:
# Defining four different filters,
# all of which are linear combinations of a 'base_filter' defined above

filters_A = get_filters(filter_A)
filters_B = get_filters(filter_B)

In [ ]:
print('Filter A')
fig_a = plot_filters(filters_A)

In [ ]:
print('Filter B')
fig_b = plot_filters(filters_B)

In [ ]:
# define a network with a single cnn and four filters

class network(nn.Module):
    
    def __init__(self, weight):
        
        super(network, self).__init__()
        
        # initialize the weights of the convolutional layer to be the weights of the four filters 
        
        k_height, k_width = weight.shape[2:]
        
        # assumes there are four grayscale filters
        
        self.conv = nn.Conv2d(1, 4, kernel_size=(k_height, k_width), bias=False)
        self.conv.weight = torch.nn.Parameter(weight)
        
    def forward(self, x):
        
        # compute the output of the cnn
        
        # pre- and post-activation
        
        conv_x = self.conv(x)
        activated_x = F.relu(conv_x)
        
        # returns both layers
        return conv_x, activated_x

filters = [filters_A, filters_B]

models = []
for f in filters:
    
    # instantiate the model and set the weights
    weight = torch.from_numpy(f).unsqueeze(1).type(torch.FloatTensor)
    model = network(weight)
    
    # print out the layers of the network
    #print(model)
    
    models.append(model)
    
print(models)

In [ ]:
# plot original image
plt.imshow(gray_img, cmap='gray')

print('Filter A')
fig_a = plot_filters(filters_A)

print('Filter B')
fig_b = plot_filters(filters_B)

# convert the image into an input Tensor
gray_img_tensor = torch.from_numpy(gray_img).unsqueeze(0).unsqueeze(1)

# get the convolutional layer (pre and post activation)
for m in models:
    
    # get the convolutional layer (pre and post activation)
    conv_layer, activated_layer = m(gray_img_tensor)
    
    # visualize the output of a conv layer
    viz_layer(conv_layer)
    
    # visualize the output of an activated conv layer
    viz_layer(activated_layer)